# Medical Appointment No Shows Capstone

# 5. MODELING

In [117]:
import pandas as pd
import numpy as np
import seaborn as sns

# plotting modules
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

import os, sys
import warnings
warnings.filterwarnings('always') 
%matplotlib inline

In [3]:
#SKLEARN 

#PREPROCESSING 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.preprocessing import OneHotEncoder

#MODEL_SELECTION
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.tree import DecisionTreeClassifier

#LINEAR_MODEL
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import Lasso, Ridge


#ENSEMBLE 
from sklearn.ensemble import RandomForestClassifier

#SVM
from sklearn.svm import SVC

from sklearn.neighbors import KNeighborsClassifier

#METRICS
from sklearn.metrics import r2_score 
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import classification_report, confusion_matrix


#PIPELINE 
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline as mpl

In [4]:
#SCIPY
from scipy.stats import randint

## Load Data

In [120]:
path = 'data/step2_output.csv'
df = pd.read_csv(path, index_col=None, parse_dates=["ScheduledDay","AppointmentDay"]) 
df.sample()

,PatientId,AppointmentId,Gender,ScheduledDay,AppointmentDay,Age,Neighborhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSSent,NoShow,WaitingDays
87083,3531739148755,5535535,F,2016-04-01 07:34:50,2016-06-01,64,JARDIM CAMBURI,0,0,0,0,0,1,No,60


In [121]:
#df_1
path = 'data/df_1_output.csv'
df_1 = pd.read_csv(path, index_col=None) 
df_1.sample()

,Gender,Neighborhood,NoShow
31428,0,13,1


In [122]:
#df_2
path = 'data/df_2_output.csv'
df_2 = pd.read_csv(path, index_col=None) 
df_2.sample()

,Gender,ScheduledDay,AppointmentDay,Age,Neighborhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSSent,NoShow,WaitingDays
90330,1,2016-06-02 08:07:00,2016-06-02 00:00:00,14,44,0,0,0,0,0,0,0,-1


In [123]:
path = 'data/df_3_output.csv'
df_3 = pd.read_csv(path, index_col=None) 
df_3.head()

,Gender,Neighborhood,NoShow,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSSent
0,0,38,0,0,1,0,0,0,0
1,1,38,0,0,0,0,0,0,0
2,0,44,0,0,0,0,0,0,0
3,0,53,0,0,0,0,0,0,0
4,0,38,0,0,1,1,0,0,0


In [124]:
path = 'data/df_gd_output.csv'
df_gd = pd.read_csv(path, index_col=None) 
df_gd.sample()

,ScheduledDay,AppointmentDay,Age,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSSent,WaitingDays,...,N_SOLON BORGES,N_SÃO BENEDITO,N_SÃO CRISTÓVÃO,N_SÃO JOSÉ,N_SÃO PEDRO,N_TABUAZEIRO,N_UNIVERSITÁRIO,N_VILA RUBIM,Show_No,Show_Yes
62430,2016-05-19 09:37:21,2016-05-19 00:00:00,68,0,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,1,0


## Sklearn's OneHotEncoder

In [125]:
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('NoShow', OneHotEncoder(), [0])], remainder = 'passthrough')

In [1]:
'''
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore') 

#ohe.fit(df_1['Neighborhood'])

transformed = ohe.fit_transform(df_1["Gender"].to_numpy().reshape(1, -1))
#transformed = ohe.transform(df_1["Gender"].toarray().reshape(-1, 1))



pd.concat([df_1.drop("Gender", 1),
          pd.DataFrame(transformed)], axis=1).reindex()
          '''

'\nohe = OneHotEncoder(sparse=False, handle_unknown=\'ignore\') \n\n#ohe.fit(df_1[\'Neighborhood\'])\n\ntransformed = ohe.fit_transform(df_1["Gender"].to_numpy().reshape(1, -1))\n#transformed = ohe.transform(df_1["Gender"].toarray().reshape(-1, 1))\n\n\n\npd.concat([df_1.drop("Gender", 1),\n          pd.DataFrame(transformed)], axis=1).reindex()\n          '

## K-NEAREST NEIGHBORS
- Model That classifies data based on its distance to training set data. A new data point is assigned a label based on the class that the majority of surrounding data points belong to. 
- “The k-nearest neighbors algorithm (KNN) is a non-parametric method used for classification and regression. In both cases, the input consists of the k closest training examples in the feature space. The output depends on whether k-NN is used for classification or regression”-Wikipedia

In [ ]:
# KNN Object
knn = KNeighborsClassifier(n_neighbors=5, 
                           weights='uniform', algorithm='auto', 
                           leaf_size=30, p=2, metric='minkowski', 
                           metric_params=None, n_jobs=None) 

X = df_1.drop(['NoShow'], axis = 1)
y = df_1['NoShow'] 

# Create training and test sets
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size = 0.2, random_state=42)

# Fit the classifier to the data
model = knn.fit(X_train, y_train)  

# Predict the labels for the training data X

y_pred = model.predict(X_test)  

#Print Classification Report
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

acc_knn = round(knn.score(X_test, y_test) * 100, 2)
print(acc_knn)

In [ ]:
# Setup arrays to store train and test accuracies
neighbors = np.arange(1, 9)
train_accuracy = np.empty(len(neighbors))
test_accuracy = np.empty(len(neighbors))

# Loop over different values of k
for i, k in enumerate(neighbors):
    # Setup a k-NN Classifier with k neighbors: knn
    knn = KNeighborsClassifier(n_neighbors=k)

    # Fit the classifier to the training data
    knn.fit(X_train, y_train)
    
    #Compute accuracy on the training set
    train_accuracy[i] = knn.score(X_train,y_train)

    #Compute accuracy on the testing set
    test_accuracy[i] = knn.score(X_test, y_test)

# Generate plot
plt.title('k-NN: Varying Number of Neighbors')
plt.plot(neighbors, test_accuracy, label = 'Testing Accuracy')
plt.plot(neighbors, train_accuracy, label = 'Training Accuracy')
plt.legend()
plt.xlabel('Number of Neighbors')
plt.ylabel('Accuracy')
plt.show()


## 1. Logistic Regression Model
Logistic regression is a supervised learning classification algorithm used to predict the probability of a target variable. The nature of target or dependent variable is dichotomous, which means there would be only two possible classes.

In [ ]:
X = df_1.drop('NoShow', axis='columns')
y = df_1.NoShow

print(X.shape)
print(y.shape)

In [ ]:
# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Create training and test sets
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size = 0.4, random_state=42)

# Create logistic regression
logreg = LogisticRegression(solver='lbfgs')

# Train the model using the training sets
logreg.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = logreg.predict(X_test)

# Compute and print the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
# Compute 5-fold cross-validation scovres: cv_scores
cv_scores = cross_val_score(logreg, X, y, cv=5, scoring='accuracy').mean()
print(cv_scores*100)
print("Average 5-Fold CV Score: {}".format(np.mean(cv_scores)))

#### Plot ROC curve

In [ ]:
# Compute predicted probabilities: y_pred_prob
y_pred_prob = logreg.predict_proba(X_test)[:,1]

# Generate ROC curve values: fpr, tpr, thresholds
fpr,tpr, thresholds = roc_curve(y_test,y_pred_prob)

# Plot ROC curve
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr,tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

#### AUC Computation for the Logistic Regression Model

In [105]:
# Compute predicted probabilities: y_pred_prob
y_pred_prob = logreg.predict_proba(X_test)[:,1]

# Compute and print AUC score
print("AUC: {}".format(roc_auc_score(y_test,y_pred_prob)))

# Compute cross-validated AUC scores: cv_auc
cv_auc = cross_val_score(logreg, X,y,scoring='roc_auc', cv=5)

# Print list of AUC scores
print("AUC scores computed using 5-fold cross-validation: {}".format(cv_auc))

AUC: 0.5109713838676995
AUC scores computed using 5-fold cross-validation: [0.5123243  0.51735173 0.46895078 0.51135791 0.50482371]


### Hyperparameter tuning with GridSearchCV

In [114]:
# Setup the hyperparameter grid
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space }

# Instantiate a logistic regression classifier: logreg
logreg = LogisticRegression()

# Instantiate the GridSearchCV object: logreg_cv
logreg_cv = GridSearchCV(logreg, param_grid,cv=5)

# Fit it to the data
logreg_cv.fit(X,y)

# Print the tuned parameters and score
print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_)) 
print("Best score is {}".format(logreg_cv.best_score_))

Tuned Logistic Regression Parameters: {'C': 1e-05}
Best score is 0.7980944108791878


In [ ]:
# Setup the parameters and distributions to sample from: param_dist
param_dist = {"max_depth": [3, None],
              "max_features": randint(1, 9),
              "min_samples_leaf": randint(1, 9),
              "criterion": ["gini", "entropy"]}

# Instantiate a Decision Tree classifier: tree
tree = DecisionTreeClassifier()

# Instantiate the RandomizedSearchCV object: tree_cv
tree_cv = RandomizedSearchCV(tree, param_dist, cv=5)

# Fit it to the data
tree_cv.fit(X,y)

# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_))
print("Best score is {}".format(tree_cv.best_score_))

In [31]:


# Create standardizer
standardizer = StandardScaler()

# Fit standardizer to training set
standardizer.fit(X_train)

# Apply to both training and test sets
#X_train_std = standardizer.transform(X_train)
#X_test_std = standardizer.transform(X_test)


#acc_log = round(logreg.score(X_train, y_train) * 100, 2)

In [33]:


# Compute and print R^2 and RMSE
print("R^2: {}".format(logit.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))

79.80944108791877
Average 5-Fold CV Score: 0.7980944108791878
R^2: 0.799488768747031
Root Mean Squared Error: 0.4477848046249102


In [34]:
explained_variance_score(y_test, y_pred)

0.0

In [35]:
mean_absolute_error(y_test, y_pred )*100

20.0511231252969

In [36]:
# Perform 3-fold CV
cvscores_3 = cross_val_score(logit, X, y, cv=3)
print(np.mean(cvscores_3))

# Perform 10-fold CV
cvscores_10 = cross_val_score(logit,X,y,cv=10)
print(np.mean(cvscores_10))

0.7980944108146258
0.7980944114499835


In [ ]:
df_ap = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
df_ap.head()

In [ ]:
# Import Lasso
from sklearn.linear_model import Lasso

# Instantiate a lasso regressor: lasso
lasso = Lasso(alpha=0.4,normalize=True)

# Fit the regressor to the data
lasso.fit(X_train,y_train)

# Compute and print the coefficients
lasso_coef = lasso.fit(X_train,y_train).coef_
print(lasso_coef)

# Plot the coefficients
plt.plot(range(len(df_1)), lasso_coef)
plt.xticks(range(len(df_1)), df_1.values, rotation=60)
plt.margins(0.02)
plt.show()


In [ ]:
y.shape

In [ ]:
# Compute and print the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Print the accuracy from the testing data.
print(accuracy_score(logit.predict(X_test), y_test))

In [ ]:
# Function to Calculate r-squared score
def custom_metric(y_test, y_pred):
    r2 = r2_score(y_test, y_pred)
    return r2

In [ ]:
# Make scorer and define that higher scores are better
score = make_scorer(custom_metric, greater_is_better=True)

In [ ]:
score(logit, X_test, y_test)

In [ ]:
# Cross-validate model using precision
cross_val_score(logit, X, y, cv=5,scoring="precision").mean()

In [ ]:
# Print the accuracy from the testing data.
print(accuracy_score(logit.predict(X_test), y_test))

In [ ]:


# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled = scaler.transform(X)

y = y.ravel()


In [ ]:
results = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 
              'Decision Tree'],
    'Score': [acc_linear_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_decision_tree]})
result_df = results.sort_values(by='Score', ascending=False)
result_df = result_df.set_index('Score')
result_df.head(9)

### Reshape

In [ ]:


# Print the dimensions of X and y before reshaping
print("Dimensions of y before reshaping: {}".format(y.shape))
print("Dimensions of X before reshaping: {}".format(X.shape))

# Reshape X and y
y = y.reshape(-1,1)
X = X.reshape(-1,1)

# Print the dimensions of X and y after reshaping
print("Dimensions of y after reshaping: {}".format(y.shape))
print("Dimensions of X after reshaping: {}".format(X.shape))


In [ ]:

# Setup the parameters and distributions to sample from: param_dist
param_dist = {"max_depth": [3, None],
              "max_features": randint(1, 9),
              "min_samples_leaf": randint(1, 9),
              "criterion": ["gini", "entropy"]}

# Instantiate a Decision Tree classifier: tree
tree = DecisionTreeClassifier()

# Instantiate the RandomizedSearchCV object: tree_cv
tree_cv = RandomizedSearchCV(tree, param_dist, cv=5)

# Fit it to the data
tree_cv.fit(X,y)

# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_))
print("Best score is {}".format(tree_cv.best_score_))

In [ ]:
# Instantiate a ridge regressor: ridge
ridge = Ridge(alpha =.5, normalize=True)

# Perform 5-fold cross-validation: ridge_cv
ridge_cv = cross_val_score(ridge,X,y,cv=5)

# Print the cross-validated scores
print(ridge_cv)


## 2. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42).fit(X_train, y_train)
model.score(X_test,y_test)

In [ ]:

confusion_matrics(y_test, model.predict(X_test))

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [10,200],
    'max_features': ['auto','sqrt','log2',0.5],
}

gs = GridSearchCV(estimator=model, param_grid=param_grid,\
                 scoring='recall',n_jobs=-1)
gs.fit(X_train,y_train)
recall_score(y_test,gs.best_estimator_.predict(X_test))

In [ ]:
from sklearn.datasets import make_classification

from sklearn.metrics import roc_curve, roc_auc_score, \
    precision_recall_curve, auc

%matplotlib inline 
%config InlineBackend.figure_format = 'svg'
X,y = make_classification(n_samples=10000, n_classes=2, \
                         weights =[0.95,0.05],random_state=42)
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=.2,\
                                                  random_state=2)
model = LogisticRegression().fit(X_train,y_train)

probs = model.predict_proba(X_test)
probs = probs[:,1]
print(probs)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train, targets,\
                                              test_size=0.2, \
                                              random_state=42)
model=RandomForestRegressor(max_features=.5,n_estimators=20)
model.fit(X_train,y_train)
model.score(X_test,y_test)

In [ ]:
from sklearn.svm import LinearSVC

# Construct data for the model
X = df[['Age', 'Neighborhood']]
y = df[['Age']]

model = LinearSVC() 
#Fit X(Training Data), y(Labels for each datapoint)
model.fit(X,y)

#Investigate what kind of patterns the data found
model.coef_

# Predict with the model
predictions = model.predict(X_test)
print(predictions)

# Create input array
X_predict = targets[['petal length (cm)', 'petal width (cm)']]



# Visualize predictions and actual values
plt.scatter(X_predict['petal length (cm)'], X_predict['petal width (cm)'],
            c=predictions, cmap=plt.cm.coolwarm)
plt.title("Predicted class values")
plt.show()